In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
!pip install pillow==4.1.1
%reload_ext autoreload
%autoreload

In [0]:
import torch.utils.data
from torchvision import datasets, models, transforms
import os
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision.transforms.functional as TF
import scipy.stats

In [0]:
## Random blur and random brightness adjustment to be added.
## About normalization we need to check in which values we normalize -> later
data_transforms = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])

In [0]:
def get_teacher_signal(features, img_name, width, height, sigma=4, downsample=4):
        f = features[(features['image_file']==img_name) & (features.label.isin(objects_to_look))]
        print(width, height)
        """Creates teacher signal for the image."""
        signal = torch.zeros((height, width))

        for i in range(len(f)):
            temp = f.iloc[i]
            xmin = temp['xmin']
            xmax = temp['xmax']
            ymin = temp['ymin']
            ymax = temp['ymax']
            
            c_x = int((xmax + xmin) / 2)
            c_y = int((ymax + ymin) / 2)

            for y in range(ymin, ymax + 1):
                for x in range(xmin, xmax + 1):
                    signal[y, x] += scipy.stats.multivariate_normal.pdf([y, x], [c_y, c_x], [sigma, sigma])
      

        # downsample
        signal = signal[::downsample, ::downsample]
        return signal


In [0]:
class SoccerBallDataset(Dataset):
    """Soccer Ball dataset."""

    def __init__(self, csv_file, root_dir, transform=None, objects_to_look = ['ball']):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.features = pd.read_csv(csv_file,sep=';' )
        self.root_dir = root_dir
        self.transform = transform
        self.objects_to_look = objects_to_look

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        # Loading image
        img_name = self.features.iloc[idx, 0]
        img_path = os.path.join(self.root_dir, img_name)
        image = io.imread(img_path)
        image = image.transpose(2,0,1)
        image = torch.from_numpy(image)

        if self.transform:
            image = TF.to_pil_image(image)
            image = self.transform(image)
            
        # Extracting features
        sample = {'image': image, 'features': self.get_teacher_signal(img_name, image.size()[2], image.size()[1])}

        return sample
    
    def get_teacher_signal(self, img_name, width, height, sigma=4, downsample=4):
        f = self.features[(self.features['image_file']==img_name) & (self.features.label.isin(self.objects_to_look))]
        """Creates teacher signal for the image."""
        signal = torch.zeros((height, width))

        for i in range(len(f)):
            temp = f.iloc[i]
            xmin = temp['xmin']
            xmax = temp['xmax']
            ymin = temp['ymin']
            ymax = temp['ymax']
            
            c_x = int((xmax + xmin) / 2)
            c_y = int((ymax + ymin) / 2)

            for y in range(ymin, ymax + 1):
                for x in range(xmin, xmax + 1):
                    signal[y, x] += scipy.stats.multivariate_normal.pdf([y, x], [c_y, c_x], [sigma, sigma])
      

        # downsample
        signal = signal[::downsample, ::downsample]
        return signal


In [0]:
# upload files if google colab !mkdir data/train
dataset = SoccerBallDataset('data/data_10.csv', 'data/train', transform=data_transforms)
dataloader = DataLoader(dataset, batch_size=4,
                        shuffle=True, num_workers=2)

In [155]:
# Just testing 
for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, sample_batched['image'].size(),
          sample_batched['features'].size())
    if(i_batch > 1):
        break

0 torch.Size([4, 3, 480, 640]) torch.Size([4, 120, 160])
1 torch.Size([4, 3, 480, 640]) torch.Size([4, 120, 160])
2 torch.Size([4, 3, 480, 640]) torch.Size([4, 120, 160])
